In [1]:
import pathlib

import numpy as np
import pandas as pd
import umap

In [2]:
# set paths
# input path
data_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/test_run/CLS_features/CLS_features_annotated.parquet"
).resolve(strict=True)
# output path
output_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/test_run/CLS_features/CLS_features_annotated_umap.csv"
).resolve()

In [3]:
# load in data
cls_df = pd.read_parquet(data_path)
cls_df.head()

,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_Channel,Metadata_Cell_id,cls_0,cls_1,cls_10,cls_100,...,cls_90,cls_91,cls_92,cls_93,cls_94,cls_95,cls_96,cls_97,cls_98,cls_99
0,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,100,0.031278,-0.054149,0.032339,0.016466,...,0.103771,-0.002917,-0.088505,-0.054172,0.009552,-0.053764,0.092698,0.052247,0.047284,0.025616
1,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,101,0.031099,-0.064783,0.014539,-0.002247,...,0.082508,-0.004323,-0.041911,-0.060461,0.007774,-0.065580,0.110302,-0.000034,0.024379,0.019590
2,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,102,0.038233,-0.058997,0.013999,-0.007888,...,0.090952,-0.004582,-0.037789,-0.048010,0.004603,-0.058200,0.084165,0.034247,0.050790,0.051865
3,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,103,0.037757,-0.071355,0.003984,0.007385,...,0.085263,0.001482,-0.084153,-0.051854,-0.000954,-0.049379,0.102376,0.030741,0.029512,0.040069
4,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,104,0.025290,-0.051070,0.019076,-0.002388,...,0.098721,-0.009961,-0.039805,-0.040878,0.022797,-0.054397,0.045918,0.046192,0.052029,0.035139


In [4]:
# get the metadata
metadata_df = cls_df.columns[cls_df.columns.str.contains("Metadata")]
metadata_df = cls_df[metadata_df]
feature_df = cls_df.drop(metadata_df.columns, axis=1)
print(f"metadata_df shape: {metadata_df.shape}")
print(f"feature_df shape: {feature_df.shape}")

metadata_df shape: (193146, 6)
feature_df shape: (193146, 384)


In [5]:
# define the UMAP model
umap_model = umap.UMAP(
    n_components=2, random_state=0, n_neighbors=30, min_dist=0.1, metric="euclidean"
)

# fit the UMAP model
umap_embedding = umap_model.fit_transform(feature_df)
umap_embedding_df = pd.DataFrame(umap_embedding, columns=["UMAP1", "UMAP2"])
# add the metadata back
umap_embedding_df = pd.concat([metadata_df, umap_embedding_df], axis=1)
umap_embedding_df.head()

/home/lippincm/miniconda3/envs/timelapse_env/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_Channel,Metadata_Cell_id,UMAP1,UMAP2
0,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,100,-0.617407,5.035722
1,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,101,-2.464208,0.645100
2,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,102,2.444222,2.684372
3,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,103,-2.317232,1.164416
4,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,104,2.748762,6.495857


In [6]:
# save the UMAP embeddings to parquet
umap_embedding_df.to_csv(output_path)